In [232]:
import string
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import pandas as pd


def get_nba_player_stats(year,player_name):
  player_name = player_name
  player_name = player_name.translate(str.maketrans('', '', string.punctuation))
  player_name_list = player_name.split(" ",1)

  played_year = year

  base_url = f'https://www.basketball-reference.com/players/{player_name_list[1][0].lower()}/{(player_name_list[1][:5]+player_name_list[0][:2]).lower()}01.html'
  pager = 1
  while True:
    
    base_html = urlopen(base_url)


    soup = BeautifulSoup(base_html, features="lxml")
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="per_game") 

    years_played = []
    if table != None:
      for row in table.tbody.find_all('tr'):
        year_col = row.find_all('a')
        if(year_col != []):
          years_played.append(year_col[0].text)

        
    if any(played_year in year for year in years_played) == False:
      pager += 1
      base_url = f'https://www.basketball-reference.com/players/{player_name_list[1][0].lower()}/{(player_name_list[1][:5]+player_name_list[0][:2]).lower()}0{pager}.html'
      
    else:
      break
    break



  base_url = base_url.replace('.html','')
  base_url = base_url + f'/gamelog/{played_year}'
  base_html = urlopen(base_url)
  soup = BeautifulSoup(base_html, features="lxml")

  year_table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=='pgl_basic') 
  
  col_names_list = ['G','Date','Age','Tm','junk','Opp','Game_Result','GS','MP','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','GmSc','+/-']
  stats_dict = {}


  if year_table != None:
    for i,col_name in enumerate(col_names_list):
      val = []
      player_name_list = []
      for row in year_table.tbody.find_all('tr'):
            columns = row.find_all('td')
            if(columns != [] and len(columns) > 8):
                if (columns[i].text == ''):
                  val.append('NA')
                else:
                  val.append(columns[i].text)
                  player_name_list.append(player_name)
      stats_dict['Player Name'] = player_name_list
      stats_dict[col_name] = val
    return pd.DataFrame(stats_dict)
  else: 
    pass


In [233]:
def get_all_nba_player_df(player_names,years):
  df_list = []
  for player_name in player_names:
    for year in years:
      df_list.append(get_nba_player_stats(year,player_name))
      print(f'Collected {player_name} {year} data')
  print('Concatinated all the data')
  return pd.concat(df_list, axis=0, ignore_index=True)


In [234]:
player_names = ["Bradley Beal",
"DeMar DeRozan",
"Klay Thompson",
"Buddy Hield",
"Justin Holiday",
"Donovan Mitchell",
"Evan Fournier",
"Josh Richardson",
"Jrue Holiday",
"J.J. Redick",
"CJ McCollum",
"Luka Doncic",
"Bryn Forbes",
"Joe Harris",
"Jeremy Lamb",
"Devin Booker",
"Danny Green",
"Jordan Clarkson",
"Marcus Smart",
"Zach LaVine",
"Eric Gordon",
"Terrence Ross",
"Tim Hardaway Jr.",
"Kevin Huerter",
"Garrett Temple",
"Kentavious Caldwell-Pope",
"Austin Rivers",
"Damyean Dotson",
"Lou Williams",
"Josh Jackson"]

years = ['2016','2017','2018','2019','2020']

all_player_data = get_all_nba_player_df(player_names,years)

Collected Bradley Beal 2016 data
Collected Bradley Beal 2017 data
Collected Bradley Beal 2018 data
Collected Bradley Beal 2019 data
Collected Bradley Beal 2020 data
Collected DeMar DeRozan 2016 data
Collected DeMar DeRozan 2017 data
Collected DeMar DeRozan 2018 data
Collected DeMar DeRozan 2019 data
Collected DeMar DeRozan 2020 data
Collected Klay Thompson 2016 data
Collected Klay Thompson 2017 data
Collected Klay Thompson 2018 data
Collected Klay Thompson 2019 data
Collected Klay Thompson 2020 data
Collected Buddy Hield 2016 data
Collected Buddy Hield 2017 data
Collected Buddy Hield 2018 data
Collected Buddy Hield 2019 data
Collected Buddy Hield 2020 data
Collected Justin Holiday 2016 data
Collected Justin Holiday 2017 data
Collected Justin Holiday 2018 data
Collected Justin Holiday 2019 data
Collected Justin Holiday 2020 data
Collected Donovan Mitchell 2016 data
Collected Donovan Mitchell 2017 data
Collected Donovan Mitchell 2018 data
Collected Donovan Mitchell 2019 data
Collected Do

In [235]:
all_player_data

,Player Name,G,Date,Age,Tm,junk,Opp,Game_Result,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,Bradley Beal,1,2015-10-28,22-122,WAS,@,ORL,W (+1),1,37:36,...,3,6,0,0,1,4,2,24,12.0,+4
1,Bradley Beal,2,2015-10-30,22-124,WAS,@,MIL,W (+5),1,29:43,...,2,3,3,0,0,5,5,26,14.4,+9
2,Bradley Beal,3,2015-10-31,22-125,WAS,NA,NYK,L (-7),1,37:12,...,4,4,0,2,0,1,4,26,14.1,-9
3,Bradley Beal,4,2015-11-04,22-129,WAS,NA,SAS,W (+3),1,36:16,...,3,5,4,3,0,1,3,25,19.9,+10
4,Bradley Beal,5,2015-11-06,22-131,WAS,@,BOS,L (-20),1,30:36,...,5,5,2,1,1,2,1,24,19.3,-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9175,Josh Jackson,18,2020-03-10,23-029,MEM,NA,ORL,L (-5),0,19:53,...,1,1,4,1,0,4,5,15,9.1,+2
9176,Josh Jackson,19,2020-07-31,23-172,MEM,@,POR,L (-5),0,13:12,...,6,6,2,0,0,1,0,8,8.2,-14
9177,Josh Jackson,20,2020-08-02,23-174,MEM,NA,SAS,L (-2),0,5:08,...,0,0,1,0,0,0,1,0,0.3,+4
9178,Josh Jackson,21,2020-08-07,23-179,MEM,NA,OKC,W (+29),0,3:34,...,2,2,0,0,0,2,0,0,-1.4,+2
